In [54]:
import boto3
import os
import pandas as pd
import yfinance as yf
import json
from datetime import datetime
from dotenv import load_dotenv

Configurations

In [55]:
load_dotenv("enviroment/.env")

aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")
bucket_name = "silver-finance-data"

In [56]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION"),
)


Functions

In [57]:
def put_json_yfinance_aws(s3_client, ticker, ticker_label, start_date, end_date,bucket_name):
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data = data.copy()
    data = data.reset_index(drop=False)
    for index, rows in data.iterrows():
        date = rows['Date']
        close = round(rows['Close'],3)
        high = round(rows['High'],3)
        low = round(rows['High'],2)
        open = round(rows['Open'],2)
        formatted_key_date = date.strftime("%Y_%m_%d")
        json_data = {
            "date" : formatted_key_date,
            "close" : close,
            "high" : high,
            "low" : low,
            "open" : open
        }
        json_data = json.dumps(json_data, indent=4)
        s3_object_name = ticker_label + "/" + formatted_key_date + "_" + ticker_label
        s3_client.put_object(
            Bucket = bucket_name,
            Key = s3_object_name,
            Body=json_data,
            ContentType="application/json"
            )
        print(s3_object_name)


    return data

In [58]:
ticker = '^BVSP'
ticker_label = ticker.replace('^','')
start_date = '1900-01-01'
end_date = '2024-09-01'
put_json_yfinance_aws(s3_client,ticker, ticker_label, start_date, end_date, 'silver-finance-data')

[*********************100%***********************]  1 of 1 completed

BVSP/1993_04_27_BVSP
BVSP/1993_04_28_BVSP
BVSP/1993_04_29_BVSP
BVSP/1993_04_30_BVSP
BVSP/1993_05_03_BVSP
BVSP/1993_05_04_BVSP
BVSP/1993_05_05_BVSP
BVSP/1993_05_06_BVSP
BVSP/1993_05_07_BVSP
BVSP/1993_05_10_BVSP
BVSP/1993_05_11_BVSP
BVSP/1993_05_12_BVSP
BVSP/1993_05_13_BVSP
BVSP/1993_05_14_BVSP
BVSP/1993_05_17_BVSP
BVSP/1993_05_18_BVSP
BVSP/1993_05_19_BVSP
BVSP/1993_05_20_BVSP
BVSP/1993_05_21_BVSP
BVSP/1993_05_24_BVSP
BVSP/1993_05_25_BVSP
BVSP/1993_05_26_BVSP
BVSP/1993_05_27_BVSP
BVSP/1993_05_28_BVSP
BVSP/1993_05_31_BVSP
BVSP/1993_06_01_BVSP
BVSP/1993_06_02_BVSP
BVSP/1993_06_03_BVSP
BVSP/1993_06_04_BVSP
BVSP/1993_06_07_BVSP
BVSP/1993_06_08_BVSP
BVSP/1993_06_09_BVSP
BVSP/1993_06_11_BVSP
BVSP/1993_06_14_BVSP
BVSP/1993_06_15_BVSP
BVSP/1993_06_16_BVSP
BVSP/1993_06_17_BVSP
BVSP/1993_06_18_BVSP
BVSP/1993_06_21_BVSP
BVSP/1993_06_22_BVSP
BVSP/1993_06_23_BVSP
BVSP/1993_06_24_BVSP
BVSP/1993_06_25_BVSP
BVSP/1993_06_28_BVSP
BVSP/1993_06_29_BVSP
BVSP/1993_06_30_BVSP
BVSP/1993_07_01_BVSP
BVSP/1993_07_

Price,Date,Close,High,Low,Open,Volume
0,1993-04-27,24.500000,25.400000,24.500000,24.799999,0
1,1993-04-28,24.299999,24.600000,23.700001,24.500000,0
2,1993-04-29,23.700001,24.799999,23.700001,24.299999,0
3,1993-04-30,24.100000,24.200001,23.700001,23.700001,0
4,1993-05-03,24.100000,24.400000,23.799999,24.100000,0
...,...,...,...,...,...,...
7757,2024-08-26,136889.000000,137013.000000,135596.000000,135608.000000,7560000
7758,2024-08-27,136776.000000,137213.000000,136664.000000,136888.000000,6745900
7759,2024-08-28,137344.000000,137469.000000,135746.000000,136776.000000,7457100
7760,2024-08-29,136041.000000,137370.000000,135858.000000,137349.000000,8385100
